In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
import os, shutil
import keras
import numpy as np
import tensorflow as tf

In [ ]:
!unzip /content/drive/My\ Drive/data.zip 

In [8]:
base_dir = '/content//data' # CHANGE

base_dir = os.getcwd()
base_dir = os.path.join(base_dir, '..')
base_dir = os.path.join(base_dir, 'data')

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

In [9]:
from tensorflow.keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    val_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32)
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(48, 48),
    label_mode="categorical",
    color_mode='grayscale',
    batch_size=32)

Found 21685 files belonging to 7 classes.
Found 5906 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [10]:
for images, labels in train_dataset.take(1):
    print(images.shape)

(32, 48, 48, 1)


In [ ]:
from tensorflow import keras

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def build_model(n_hidden=1, n_neurons=128, learning_rate=3e-3, input_shape=(48, 48, 1)):
    model = keras.models.Sequential()
    model.add(keras.layers.Rescaling(1./255))
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

    return model

In [ ]:
from tensorflow import keras
from sklearn.base import BaseEstimator, RegressorMixin

# Modify the KerasRegressorWrapper class to accept hyperparameters
class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_hidden=1, n_neurons=200, learning_rate=1e-3):
        self.n_hidden = n_hidden
        self.n_neurons = n_neurons
        self.learning_rate = learning_rate

    def fit(self, X, y, **kwargs):
        self.model = build_model(self.n_hidden, self.n_neurons, self.learning_rate)
        self.model.fit(X, y, **kwargs)
        return self
    
    def fit(self, train, **kwargs):
        self.model = build_model(self.n_hidden, self.n_neurons, self.learning_rate)
        self.model.fit(train, **kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Create an instance of the KerasRegressorWrapper
keras_reg = KerasRegressorWrapper()

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_with_parameter_search.h5",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(patience=10)
]

param_distribs = {
    # fill in code
    "n_hidden": [0,1,2,3],
    "n_neurons": np.arange(100, 300).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, verbose=2, n_jobs=-1)
rnd_search_cv.fit(train_dataset, epochs=100,
                  validation_data=validation_dataset,
                  callbacks=callbacks)

In [ ]:
rnd_search_cv.best_params_

In [ ]:
model = rnd_search_cv.best_estimator_.model
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator()
model = keras.models.load_model("convnet_with_parameter_search.h5")

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(218, 178),
        batch_size=20,
        class_mode='categorical')

test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('test acc:', test_acc)